In [15]:
import h5py
import numpy as np

ens='cD96'
ensemble='cD211.054.96'

cfg2new=lambda cfg: {'r0':'a','r1':'b','r2':'c','r3':'d'}[cfg[-2:]]+cfg[:4]
cfg2old=lambda cfg: cfg[1:]+'_'+{'a':'r0','b':'r1','c':'r2','d':'r3'}[cfg[0]]

path='/p/project/pines/li47/code/projectData/NST_b-discNJN/{}/data_merge/NST_b.h5_{}'.format(ensemble,ens)
path2='/p/project/pines/li47/code/projectData/NST_b-discNJN-old/data_merge/NST_b.h5_main'
path3='/p/project/pines/li47/code/projectData/NST_b-discNJN/connFromOthers/cD211.054.96/charges_8_20_127srcs.h5'
path4='/p/project/pines/li47/code/projectData/NST_b-discNJN/connFromOthers/cD211.054.96/charges_22_to_26_112srcs.h5'
outfile='dat/NST_b_{}.h5'.format(ens)
with h5py.File(path) as f, h5py.File(outfile,'w') as fw, h5py.File(path3) as f3, h5py.File(path4) as f4:
    cfgs1=[cfg.decode() for cfg in f['cfgs']]
    cfgs3=[cfg2new(cfg) for cfg in f3['gS/up/dt8'].keys()]
    cfgs=list(set(cfgs1).intersection(set(cfgs3)))
    cfgs.sort()
    cfgs_old=[cfg2old(cfg) for cfg in cfgs]
    print(len(cfgs),len(cfgs1),len(cfgs3))
    
    dic={}
    for i,cfg in enumerate(cfgs1):
        dic[cfg]=i
    cfgs1_map=[dic[cfg] for cfg in cfgs]    

    key='VEV/j/data/j+'; fw.create_dataset(key,data=f[key][cfgs1_map])
    key='VEV/j/inserts'; fw.create_dataset(key,data=f[key])
    key='cfgs'; fw.create_dataset(key,data=cfgs)
    key='diags/N/data/N2_N2'; fw.create_dataset(key,data=f[key][cfgs1_map])
    key='diags/N/opabs'; fw.create_dataset(key,data=f[key])

    opabs=[opab.decode() for opab in f['diags/N-j/opabs']]; dic={}
    for i,opab in enumerate(opabs):
        dic[opab]=i
    opabs_new=['g;0,0,0;G1g;a;l1;N_g;0,0,0;G1g;a;l1;N','g;0,0,0;G1g;a;l2;N_g;0,0,0;G1g;a;l2;N']
    opabs_map=[dic[opab] for opab in opabs_new]
    
    inserts=[insert.decode() for insert in f['diags/N-j/inserts']]; dic={}
    for i,insert in enumerate(inserts):
        dic[insert]=i
    inserts_new=['id']
    inserts_map=[dic[insert] for insert in inserts_new]
    
    tfs=[int(key.split('_')[-1]) for key in f['diags/N-j/data'].keys()]
    tfs=list(set(tfs))
    tfs.sort()
    key='diags/N-j/opabs'; fw.create_dataset(key,data=opabs_new)
    key='diags/N-j/inserts'; fw.create_dataset(key,data=inserts_new)
    for tf in tfs:
        key='diags/N-j/data/N2_j+_N2_deltat_{}'.format(tf)
        t=f[key][:]
        t=t[cfgs1_map]
        t=t[:,:,opabs_map]
        t=t[:,:,:,inserts_map]
        fw.create_dataset(key,data=t)

    for file in [f3,f4]:
        tfs=[int(key[2:]) for key in file['gS/up'].keys()]
        tfs=list(set(tfs))
        tfs.sort()
        for tf in tfs:
            tup=np.array([file['gS/up/dt{}'.format(tf)][cfg][:] for cfg in cfgs_old])
            tdn=np.array([file['gS/up/dt{}'.format(tf)][cfg][:] for cfg in cfgs_old])
            
            key='diags/NJN/data/gS_j+_deltat_{}'.format(tf)
            tP=-(tup+tdn)
            fw.create_dataset(key,data=tP)

            key='diags/N_correlated_NJN/data/dt{}_twop'.format(tf)
            t2pt=np.array([file['dt{}_twop'.format(tf)][cfg][()] for cfg in cfgs_old])
            fw.create_dataset(key,data=t2pt)
    
    # print(cfgs[0])
    print(cfgs[:6])
    t1=np.real(fw['diags/N/data/N2_N2'][2,:4,0]); t2=np.real(fw['diags/N/data/N2_N2'][2,:4,1])
    print(t1)
    print(t2)
    print((t1+t2)/2)
    print(f3['twop']['0004_r0'][:4])
    print(f4['twop']['0004_r0'][:4])

494 494 496
['a0000', 'a0002', 'a0004', 'a0006', 'a0008', 'a0010']
[2.9333905e-10 6.3708205e-10 3.9474937e-10 2.6185060e-10]
[2.8954972e-10 6.3409139e-10 3.9248127e-10 2.5916136e-10]
[2.9144437e-10 6.3558669e-10 3.9361531e-10 2.6050598e-10]
[-3.11608378e-13  6.35134174e-10  3.91448478e-10  2.60600080e-10]
[-3.40127232e-13  6.45442123e-10  4.00432007e-10  2.67093413e-10]


In [19]:
import pickle
path='/p/project/pines/li47/code/projectData/NST_b-discNJN/connFromOthers/cD211.054.96/D96_2pt_christos.pkl' 
with open(path,'rb') as f:
    dat=pickle.load(f)
print(dat.keys())
cfgs=dat['confs']
print(cfgs)
dic={}
for i,cfg in enumerate(cfgs):
    dic[cfg]=i
ind=dic['0004_r0']
print(ind)
dat['twop_mom0'].shape
ele=dat['twop_mom0'][ind,:4]
print(ele)

# for ele in dat['twop_mom0'][:,:4]:
#     print(ele-[2.9144437e-10, 6.3558669e-10, 3.9361531e-10, 2.6050598e-10])

dict_keys(['confs', 'twop_mom0', 'twop_mom1', 'A0_mom1'])
{'0190_r3', '0062_r3', '0036_r3', '0222_r2', '0122_r2', '0234_r0', '0220_r3', '0344_r0', '0136_r1', '0196_r3', '0148_r2', '0100_r1', '0126_r0', '0054_r2', '0004_r3', '0296_r0', '0156_r1', '0078_r3', '0038_r1', '0200_r3', '0030_r3', '0024_r3', '0168_r2', '0120_r3', '0162_r3', '0048_r1', '0024_r2', '0146_r2', '0126_r2', '0066_r2', '0260_r0', '0322_r0', '0060_r2', '0230_r0', '0282_r0', '0170_r1', '0028_r3', '0128_r2', '0310_r0', '0108_r3', '0102_r3', '0194_r3', '0050_r3', '0188_r2', '0012_r3', '0200_r2', '0104_r0', '0104_r1', '0076_r0', '0178_r3', '0006_r2', '0080_r3', '0244_r0', '0176_r0', '0142_r2', '0162_r2', '0072_r3', '0016_r3', '0050_r0', '0130_r2', '0118_r0', '0046_r2', '0284_r0', '0096_r1', '0218_r3', '0196_r1', '0192_r3', '0030_r0', '0210_r2', '0152_r3', '0138_r0', '0242_r0', '0140_r2', '0040_r3', '0064_r0', '0302_r0', '0092_r2', '0156_r3', '0106_r3', '0162_r0', '0226_r0', '0116_r2', '0124_r0', '0212_r2', '0312_r0', '0096_